In [0]:
%sql
DROP SCHEMA IF EXISTS cpg_industry.gold_product CASCADE;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS cpg_industry.gold_product;

In [0]:
%sql
-- CREATE TABLE cpg_industry.gold_product.product_kpi AS
WITH product_metrics AS (
    SELECT
        p.product_id,
        p.product_name,
        p.category,
        p.brand,
        ROUND(SUM(coi.quantity * coi.unit_price), 2) AS gross_revenue,
        ROUND(SUM((coi.unit_price - p.unit_price) * coi.quantity), 2) AS gross_profit,
        SUM(coi.quantity) AS sales_volume,
        ROUND(SUM((coi.unit_price - p.unit_price) * coi.quantity) / 
              NULLIF(SUM(coi.quantity * coi.unit_price),0) * 100, 2) AS gross_margin_pct,
        COUNT(DISTINCT c.consumer_id) AS total_customers,
        COUNT(DISTINCT CASE WHEN c.is_active THEN c.consumer_id END) AS retained_customers,
        ROUND(COUNT(DISTINCT CASE WHEN NOT c.is_active THEN c.consumer_id END) * 100.0 /
              NULLIF(COUNT(DISTINCT c.consumer_id),0), 2) AS churn_rate_pct
    FROM cpg_industry.silver.cpg_consumer_order_items coi
    JOIN cpg_industry.silver.cpg_product p ON coi.product_id = p.product_id
    JOIN cpg_industry.silver.cpg_consumer_order co ON coi.order_id = co.order_id
    JOIN cpg_industry.silver.cpg_consumer c ON co.consumer_id = c.consumer_id
    GROUP BY p.product_id, p.product_name, p.category, p.brand
)
SELECT * FROM product_metrics;
